In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/fonts classifiction"

/content/drive/MyDrive/fonts classifiction


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan  3 16:05 2022

@author:David BM
"""


import h5py
import numpy as np


from sklearn.model_selection import train_test_split
from tensorflow.python.keras.backend import dropout
#importing the required libraries

import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
import numpy # fix random seed for reproducibility
import os

### define the model

In [ ]:
print(len(images))

84591


In [ ]:
numpy.random.seed(123)

db=h5py.File('adaptedImage.h5', 'r')
images=list(db)
labels=list()
values=list()

# get the values and the labels  in order to split them
# to train and test groups
for i in images:
    im=db[i]
    label=im.attrs['label']
    arr=np.array(im)
    values.append(arr)
    labels.append(label)

values=np.array(values)
labels=np.array(labels)
X_train,X_test,y_train,y_test=train_test_split(values,labels,test_size=0.15)

#reshaping data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 3))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],3))

#normalizing the pixel values
X_train=X_train/255
X_test=X_test/255

#preProcessing class labels
y_train=np_utils.to_categorical(y_train,num_classes=7)
y_test=np_utils.to_categorical(y_test,num_classes=7)

#defining model
model=Sequential()
#adding convolution layer
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)))
#adding pooling layer
model.add(MaxPool2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
#adding pooling layer
model.add(MaxPool2D(2,2))
#adding convolution layer
#adding fully connected layer
model.add(Flatten())

#initialize the weights from a gaussian distribution
model.add(Dense(units=343,activation='relu'))
#avoid overfitting
model.add(Dropout(0.5))
#adding output layer
model.add(Dense(7,activation='softmax')) # '7' is the number of the labels.


In [ ]:
tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:



#compiling the model
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
#fitting the model
model.fit(X_train,y_train,epochs=20,batch_size=32,verbose=1)

model.summary()
#evaluating the model
score=model.evaluate(X_test,y_test)

print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

db.close()






```
`# _________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d_2 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2304)              0         
                                                                 
 dense_2 (Dense)             (None, 343)               790615    
                                                                 
 dropout_1 (Dropout)         (None, 343)               0         
                                                                 
 dense_3 (Dense)             (None, 7)                 2408      
                                                                 
=================================================================
Total params: 812,415
Trainable params: 812,415
Non-trainable params: 0
_________________________________________________________________
397/397 [==============================] - 1s 2ms/step - loss: 0.2553 - accuracy: 0.9188
accuracy: 91.88%
Saved model to disk`
```

